In [1]:
import hashlib
from cryptography.fernet import Fernet
def hash_password(password):
    password = password.encode()
    hashed = hashlib.sha256(password).hexdigest()
    return hashed


In [2]:
key = Fernet.generate_key()
f = Fernet(key)
def encrypt(data):
    token = f.encrypt(data)
    return token

def decrypt(token):
    text = f.decrypt(token)
    return text

In [3]:
import os
from cryptography.fernet import Fernet

def load_key():
    if os.path.exists("secret.key"):
        with open("secret.key","rb") as file:
            key = file.read()
    else:
        key = Fernet.generate_key()
        with open("secret.key","wb") as file:
            file.write(key)
    return key

key = load_key()
f = Fernet(key)


In [4]:
def get_valid_password():
    ok = False
    while ok == False:
        pw = input("Enter a password : ")
        score = check_strength(pw)
        print("Strength score =", score)

        if score >= 6:
            ok = True
        else:
            print("Password too weak, try again")

    return pw


In [5]:
def save_user_credentials(username, hashed_pw):
    with open("users.txt","a") as file:
        file.write(username + ":" + hashed_pw + "\n")


In [6]:
def check_user(Uname):
    Uname = str(Uname)

    if os.path.exists("users.txt") == False:
        return False

    with open("users.txt","r") as file:
        lines = file.readlines()

    for line in lines:
        parts = line.strip().split(":")
        if len(parts) == 2:
            if parts[0] == Uname:
                return True

    return False


In [7]:
def login():
    Uname = input("Username: ")
    Pw = input("Password: ")

    hashed = hash_password(Pw)

    if os.path.exists("users.txt") == False:
        return False, None

    with open("users.txt","r") as file:
        lines = file.readlines()

    for line in lines:
        parts = line.strip().split(":")
        if len(parts) == 2:
            if parts[0] == Uname and parts[1] == hashed:
                return True, Uname

    return False, None


In [8]:
import os

def make_user_folder(username):
    base = "users"
    folder = base + "/" + username + "/files"

    if os.path.exists(folder) == False:
        os.makedirs(folder)

    return folder


In [9]:
def save_encrypted_file(username, fname, token):
    folder = make_user_folder(username)
    path = folder + "/" + fname

    with open(path,"wb") as file:
        file.write(token)


In [10]:
def load_and_decrypt(username, fname):
    folder = "users/" + username + "/files"
    path = folder + "/" + fname

    with open(path,"rb") as file:
        token = file.read()

    text = decrypt(token)
    return text


In [11]:
def delete_file(username, fname):
    folder = "users/" + username + "/files"
    path = folder + "/" + fname

    if os.path.exists(path):
        os.remove(path)
        print("File deleted")
    else:
        print("File not found")


In [12]:
def list_files(username):
    folder = "users/" + username + "/files"

    if os.path.exists(folder) == False:
        print("No files found")
        return

    files = os.listdir(folder)
    for f in files:
        print(f)


In [13]:
def add_user():
    print("_____ ADDING NEW USER _____")
    Uname = input("Choose a username: ")

    if check_user(Uname) == True:
        print("User already exists")
        return

    print("Create a strong password")

    ok = False
    while ok == False:
        pw = get_valid_password()
        pw2 = input("Confirm password: ")

        if pw == pw2:
            ok = True
        else:
            print("Passwords do not match, try again")

    hashed_pw = hash_password(pw)
    save_user_credentials(Uname, hashed_pw)
    make_user_folder(Uname)

    print("User created")


In [14]:
def upload_file(username):
    print("_____ UPLOAD FILE _____")
    fname = input("Enter file name to store: ")

    folder = "users/" + username + "/files"
    path = folder + "/" + fname

    if os.path.exists(path):
        print("File already exists, choose another name")
        return

    text = input("Enter file contents: ")
    text = text.encode()
    token = encrypt(text)

    save_encrypted_file(username, fname, token)

    print("File encrypted and saved")


In [15]:
def view_file(username):
    print("_____ VIEW FILE _____")
    fname = input("Enter file name: ")

    try:
        data = load_and_decrypt(username, fname)
        print("Decrypted text:")
        print(data.decode())
    except:
        print("Could not open or decrypt file")


In [16]:
def delete_user_file(username):
    print("_____ DELETE FILE _____")
    fname = input("Enter file name: ")
    delete_file(username, fname)


In [17]:
def show_files(username):
    print("_____ USER FILES _____")
    list_files(username)


In [18]:
def user_menu(username):
    print("Welcome", username)

    run = True
    while run == True:
        print("")
        print("Choose an option:")
        print("1. Upload file")
        print("2. View file")
        print("3. Delete file")
        print("4. List files")
        print("5. Logout")

        choice = input("Enter choice: ")

        if choice == "1":
            upload_file(username)
        elif choice == "2":
            view_file(username)
        elif choice == "3":
            delete_user_file(username)
        elif choice == "4":
            show_files(username)
        elif choice == "5":
            run = False
        else:
            print("Invalid choice")


In [19]:
def check_strength(password):
    lower_letters = "abcdefghijklmnopqrstuvwxyz"
    upper_letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    digits = "0123456789"
    symbols = "~@#$%^&*()_+-=,./:;|"

    has_lower = False
    has_upper = False
    has_digit = False
    has_symbol = False

    repeated_chars = False
    has_sequence = False

    common_bad = ["password", "qwerty", "abc123", "111111", "123456"]

    for i in range(len(password)):
        ch = password[i]

        if ch in lower_letters:
            has_lower = True
        if ch in upper_letters:
            has_upper = True
        if ch in digits:
            has_digit = True
        if ch in symbols:
            has_symbol = True

        if i > 0 and password[i] == password[i-1]:
            repeated_chars = True

        if i >= 2:
            a = password[i-2]
            b = password[i-1]
            c = password[i]

            if a in lower_letters and b in lower_letters and c in lower_letters:
                if ord(b) == ord(a)+1 and ord(c) == ord(b)+1:
                    has_sequence = True

            if a in upper_letters and b in upper_letters and c in upper_letters:
                if ord(b) == ord(a)+1 and ord(c) == ord(b)+1:
                    has_sequence = True

            if a.isdigit() and b.isdigit() and c.isdigit():
                if int(b) == int(a)+1 and int(c) == int(b)+1:
                    has_sequence = True

    score = 0

    if len(password) >= 8:
        score += 2
    if len(password) >= 12:
        score += 1

    if has_lower: score += 2
    if has_upper: score += 2
    if has_digit: score += 2
    if has_symbol: score += 2

    if repeated_chars:
        score -= 1
    if has_sequence:
        score -= 2

    pw_lower = password.lower()
    for bad in common_bad:
        if bad in pw_lower:
            score -= 3

    if score < 0:
        score = 0
    if score > 10:
        score = 10

    return score

In [20]:
def main_menu():
    run = True
    while run == True:
        print("\n_____ MAIN MENU _____")
        print("1. Register")
        print("2. Login")
        print("3. Exit")

        choice = input("Enter choice: ")

        if choice == "1":
            add_user()

        elif choice == "2":
            print("_____ LOGIN _____")
            ok, uname = login()
            if ok == True:
                user_menu(uname)
            else:
                print("Login failed")

        elif choice == "3":
            run = False
        else:
            print("Invalid choice")


In [21]:
def gui_list_files(username):
    import tkinter as tk
    from tkinter import messagebox
    import os

    # finding file path 
    folder = os.path.join("users", username, "files")

    if not os.path.exists(folder):
        messagebox.showinfo("Files", "No files found.")
        return

    files = os.listdir(folder)

    win = tk.Toplevel()
    win.title("Your Files")
    win.geometry("300x250")

    tk.Label(win, text="Files for " + username, font=("Arial", 12)).pack(pady=5)

    if len(files) == 0:
        tk.Label(win, text="No files found").pack()
        return

    listbox = tk.Listbox(win, width=40)
    listbox.pack(pady=10)

    for f in files:
        listbox.insert(tk.END, f)


In [22]:
def gui_view_file(username):
    import tkinter as tk
    from tkinter import messagebox

    # ask for filename
    popup = tk.Toplevel()
    popup.title("View File")
    popup.geometry("260x150")

    tk.Label(popup, text="Enter file name:").pack(pady=5)
    entry = tk.Entry(popup)
    entry.pack(pady=5)

    def load():
        fname = entry.get()
        try:
            data = load_and_decrypt(username, fname)
            text = data.decode()

        
            viewer = tk.Toplevel()
            viewer.title(fname)
            viewer.geometry("360x300")

            tk.Label(viewer, text=f"Contents of {fname}", font=("Arial", 12)).pack(pady=5)

            txt = tk.Text(viewer, wrap="word")
            txt.pack(expand=True, fill="both")
            txt.insert("1.0", text)

        except:
            messagebox.showerror("Error", "Could not open or decrypt file")

        popup.destroy()

    tk.Button(popup, text="Open", width=15, command=load).pack(pady=10)


In [23]:
def gui_register():
    import tkinter as tk
    from tkinter import messagebox

    win = tk.Toplevel()
    win.title("Register New User")
    win.geometry("320x280")

    tk.Label(win, text="Choose a username:").pack(pady=5)
    entry_user = tk.Entry(win)
    entry_user.pack(pady=5)

    tk.Label(win, text="Choose a strong password:").pack(pady=5)
    entry_pass = tk.Entry(win, show="*")
    entry_pass.pack(pady=5)

    tk.Label(win, text="Confirm password:").pack(pady=5)
    entry_pass2 = tk.Entry(win, show="*")
    entry_pass2.pack(pady=5)

    def submit_register():
        username = entry_user.get().strip()
        pw1 = entry_pass.get()
        pw2 = entry_pass2.get()

        if username == "" or pw1 == "" or pw2 == "":
            messagebox.showerror("Error", "All fields are required")
            return

        if pw1 != pw2:
            messagebox.showerror("Error", "Passwords do not match")
            return

        if check_user(username):
            messagebox.showerror("Error", "User already exists")
            return

        # check password strength
        score = check_strength(pw1)
        if score < 6:
            messagebox.showerror("Error", f"Password too weak! Strength score: {score}")
            return

        # hash password and save credentials
        hashed_pw = hash_password(pw1)
        save_user_credentials(username, hashed_pw)

        # make user folder
        make_user_folder(username)

        messagebox.showinfo("Success", f"User '{username}' created successfully!")
        win.destroy()

    tk.Button(win, text="Register", width=20, command=submit_register).pack(pady=15)
    tk.Button(win, text="Cancel", width=20, command=win.destroy).pack()


In [24]:
def gui_upload_file(username):
    import tkinter as tk
    from tkinter import messagebox

    win = tk.Toplevel()
    win.title("Upload File")
    win.geometry("350x300")

    tk.Label(win, text="File Name:").pack(pady=4)
    entry_name = tk.Entry(win)
    entry_name.pack(pady=4)

    tk.Label(win, text="File Contents:").pack(pady=4)
    text_box = tk.Text(win, height=8, width=30)
    text_box.pack(pady=4)

    def save():
        fname = entry_name.get()
        contents = text_box.get("1.0", tk.END).strip()

        if fname.strip() == "":
            messagebox.showerror("Error", "File name cannot be empty")
            return

        token = encrypt(contents.encode())
        save_encrypted_file(username, fname, token)

        messagebox.showinfo("Done", "File encrypted and saved.")
        win.destroy()

    tk.Button(win, text="Upload", width=15, command=save).pack(pady=10)


In [25]:
def gui_delete_file(username):
    import tkinter as tk
    from tkinter import messagebox

    popup = tk.Toplevel()
    popup.title("Delete File")
    popup.geometry("260x150")

    tk.Label(popup, text="Enter file name:").pack(pady=5)
    entry = tk.Entry(popup)
    entry.pack(pady=5)

    def delete_now():
        fname = entry.get()
        folder = "user_data_" + username
        path = folder + "/" + fname

        import os
        if os.path.exists(path):
            os.remove(path)
            messagebox.showinfo("Done", "File deleted.")
        else:
            messagebox.showerror("Error", "File not found.")

        popup.destroy()

    tk.Button(popup, text="Delete", width=15, command=delete_now).pack(pady=10)


In [26]:
def gui_user_menu(username):
    import tkinter as tk
    from tkinter import messagebox

    win = tk.Toplevel()
    win.title(f"User Menu - {username}")
    win.geometry("330x300")

    tk.Label(win, text=f"Welcome {username}", font=("Arial", 13)).pack(pady=10)
    tk.Label(win, text="Choose an option:", font=("Arial", 11)).pack(pady=5)

    
    def run_with_inputs(func, prompts):

        answers = []

        # Ask each required input
        for p in prompts:
            popup = tk.Toplevel(win)
            popup.title("Input Required")
            popup.geometry("260x130")

            tk.Label(popup, text=p).pack(pady=5)
            entry = tk.Entry(popup)
            entry.pack(pady=5)

            def submit():
                answers.append(entry.get())
                popup.destroy()

            tk.Button(popup, text="OK", width=10, command=submit).pack(pady=5)
            popup.wait_window()

        # Fake input() to feed answers
        def fake_input(prompt=None):
            return answers.pop(0)

        real_input = __builtins__.input
        __builtins__.input = fake_input

        try:
            func(username)
        except Exception as e:
            messagebox.showerror("Error", str(e))
        finally:
            __builtins__.input = real_input

def on_upload():
    gui_upload_file(username)

def on_view():
    gui_view_file(username)

def on_delete():
    gui_delete_file(username)

def on_list():
    gui_list_files(username)

    def on_logout():
        win.destroy()

    tk.Button(win, text="1. Upload File", width=25, command=on_upload).pack(pady=6)
    tk.Button(win, text="2. View File", width=25, command=on_view).pack(pady=6)
    tk.Button(win, text="3. Delete File", width=25, command=on_delete).pack(pady=6)
    tk.Button(win, text="4. List Files", width=25, command=on_list).pack(pady=6)
    tk.Button(win, text="5. Logout", width=25, command=on_logout).pack(pady=10)


In [27]:
def gui_main_menu():
    import tkinter as tk
    from tkinter import messagebox

    BG_COLOR = "#2C3E50"
    FG_COLOR = "#ECF0F1"
    BTN_COLOR = "#2980B9"
    BTN_HOVER = "#3498DB"
    FONT_TITLE = ("Helvetica", 16, "bold")
    FONT_BTN = ("Helvetica", 11, "bold")
    PAD_Y = 8

    def style_button(btn):
        btn.config(bg=BTN_COLOR, fg=FG_COLOR, font=FONT_BTN, activebackground=BTN_HOVER, relief="raised")
        btn.bind("<Enter>", lambda e: btn.config(bg=BTN_HOVER))
        btn.bind("<Leave>", lambda e: btn.config(bg=BTN_COLOR))

    root = tk.Tk()
    root.title("MAIN MENU")
    root.configure(bg=BG_COLOR)
    root.geometry("350x300")

    tk.Label(root, text="_____ MAIN MENU _____", font=FONT_TITLE, fg=FG_COLOR, bg=BG_COLOR).pack(pady=15)

    def on_register():
        gui_register()

    btn_reg = tk.Button(root, text="1. Register", width=25, command=on_register)
    style_button(btn_reg)
    btn_reg.pack(pady=PAD_Y)

#login section
    def on_login():
        win = tk.Toplevel(root)
        win.title("LOGIN")
        win.configure(bg=BG_COLOR)
        win.geometry("350x220")

        tk.Label(win, text="Login", font=FONT_TITLE, fg=FG_COLOR, bg=BG_COLOR).pack(pady=10)
        tk.Label(win, text="Username:", fg=FG_COLOR, bg=BG_COLOR).pack(pady=PAD_Y)
        entry_user = tk.Entry(win)
        entry_user.pack()

        tk.Label(win, text="Password:", fg=FG_COLOR, bg=BG_COLOR).pack(pady=PAD_Y)
        entry_pass = tk.Entry(win, show="*")
        entry_pass.pack()

        def submit_login():
            uname_val = entry_user.get()
            pw_val = entry_pass.get()
            inputs = [uname_val, pw_val]

            def fake_input(prompt=None):
                return inputs.pop(0)

            real_input = __builtins__.input
            __builtins__.input = fake_input

            try:
                ok, username = login()
            except:
                ok, username = False, None
            finally:
                __builtins__.input = real_input

            if ok:
                messagebox.showinfo("Success", f"Login successful. Welcome, {username}!")
                win.destroy()
                gui_user_menu(username)
            else:
                messagebox.showerror("Error", "Login failed")

        btn_login = tk.Button(win, text="Login", width=20, command=submit_login)
        style_button(btn_login)
        btn_login.pack(pady=PAD_Y)

        btn_cancel = tk.Button(win, text="Cancel", width=20, command=win.destroy)
        style_button(btn_cancel)
        btn_cancel.pack(pady=PAD_Y)

    btn_login_main = tk.Button(root, text="2. Login", width=25, command=on_login)
    style_button(btn_login_main)
    btn_login_main.pack(pady=PAD_Y)

#exit option codedd here
    def on_exit():
        root.destroy()

    btn_exit = tk.Button(root, text="3. Exit", width=25, command=on_exit)
    style_button(btn_exit)
    btn_exit.pack(pady=PAD_Y+5)

    root.mainloop()


def gui_user_menu(username):
    import tkinter as tk
    from tkinter import messagebox

    BG_COLOR = "#2C3E50"
    FG_COLOR = "#ECF0F1"
    BTN_COLOR = "#2980B9"
    BTN_HOVER = "#3498DB"
    FONT_TITLE = ("Helvetica", 16, "bold")
    FONT_BTN = ("Helvetica", 11, "bold")
    PAD_Y = 8

    def style_button(btn):
        btn.config(bg=BTN_COLOR, fg=FG_COLOR, font=FONT_BTN, activebackground=BTN_HOVER, relief="raised")
        btn.bind("<Enter>", lambda e: btn.config(bg=BTN_HOVER))
        btn.bind("<Leave>", lambda e: btn.config(bg=BTN_COLOR))

    win = tk.Toplevel()
    win.title(f"User Menu - {username}")
    win.configure(bg=BG_COLOR)
    win.geometry("350x360")

    tk.Label(win, text=f"Welcome {username}", font=FONT_TITLE, fg=FG_COLOR, bg=BG_COLOR).pack(pady=15)
    tk.Label(win, text="Choose an option:", fg=FG_COLOR, bg=BG_COLOR).pack(pady=PAD_Y)

    def on_upload():
        gui_upload_file(username)

    def on_view():
        gui_view_file(username)

    def on_delete():
        gui_delete_file(username)

    def on_list():
        gui_list_files(username)

    def on_logout():
        win.destroy()

    buttons = [
        ("1. Upload File", on_upload),
        ("2. View File", on_view),
        ("3. Delete File", on_delete),
        ("4. List Files", on_list),
        ("5. Logout", on_logout)
    ]

    for text, cmd in buttons:
        b = tk.Button(win, text=text, width=25, command=cmd)
        style_button(b)
        b.pack(pady=PAD_Y)


In [ ]:
gui_main_menu()